In [ ]:
!pip install -q --upgrade unsloth datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.8/348.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.7/276.7 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [ ]:
 import unsloth
 import torch
 from trl import SFTTrainer
 from datasets import load_dataset
 from unsloth import FastLanguageModel
 from unsloth import FastLanguageModel
 from transformers import GenerationConfig
 from transformers import TrainingArguments
 from huggingface_hub import login

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
dataset_name = "5CD-AI/Vietnamese-Multi-turn-Chat-Alpaca"
raw_dataset = load_dataset(dataset_name, split="train")

README.md:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

vi_chatalpaca_cleaned.json:   0%|          | 0.00/69.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12697 [00:00<?, ? examples/s]

In [ ]:
raw_dataset[0]['conversations']

[{'from': 'human',
  'value': 'Hãy chỉnh sửa câu này để ngắn gọn hơn mà không mất đi ý nghĩa: "Trận đấu là một thất bại nặng nề mặc dù thực tế là cả đội đã tập luyện trong nhiều tuần."'},
 {'from': 'gpt',
  'value': 'Nhiều tuần huấn luyện của đội đã dẫn đến một thất bại nặng nề.'},
 {'from': 'human',
  'value': 'Bạn có thể đề xuất một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của họ trong trận đấu tiếp theo không?'},
 {'from': 'gpt',
  'value': 'Chắc chắn, đây là một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của mình trong trận đấu tiếp theo: 1. Phân tích trận đấu trước bằng cách xem lại cảnh quay trận đấu để xác định điểm yếu và các lĩnh vực cần cải thiện. 2. Tăng cường độ và sự tập trung của các buổi tập để đảm bảo mọi cầu thủ đều được chuẩn bị đầy đủ về thể chất và tinh thần. 3. Luyện tập những kỹ năng cụ thể cần cải thiện, chẳng hạn như sút bóng hoặc chuyền bóng chính xác. 4. Phát triển các chiến lược và chiến thuật của đội có tính đến điểm mạnh 

In [ ]:
MAX_SEQ_LENGTH = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=True,) # Changed from torch.bfloat16

In [ ]:
model = FastLanguageModel.get_peft_model(
    model, r=16, lora_alpha=16, lora_dropout=0,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "up_proj",
        "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 42)

Unsloth 2025.11.1 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [ ]:
SYS_INSTRUCT = "Bạn là một trợ lý AI thân thiện, hãy trả lời bằng tiếng Việt."

def convert_to_chat_format(conversations):
    messages = [{"role": "system", "content": SYS_INSTRUCT}]
    for msg in conversations:
        role = "user" if msg["from"] == "human" else "assistant"
        messages.append({"role": role, "content": msg["value"]})
    return messages

def format_prompt(example):
    messages = convert_to_chat_format(example["conversations"])
    return {
        "text": tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
    }

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        padding="max_length",
    )

dataset = raw_dataset.map(format_prompt, remove_columns=["conversations"])

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=32,
    gradient_accumulation_steps=2,
    learning_rate=1e-4,
    save_total_limit=4,
    logging_steps=20,
    output_dir="./checkpoint/llama3-1b-multi-conversation",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    save_strategy="steps",
    save_steps=50,
    report_to="none",
    remove_unused_columns=True,
    max_steps=400,
    fp16=True,
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/12697 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,697 | Num Epochs = 3 | Total steps = 400
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 2 x 1) = 64
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
20,1.823100
40,1.520800
60,1.464300
80,1.430100
100,1.392000
120,1.359400
140,1.346400
160,1.330500
180,1.326200
200,1.323000


Step,Training Loss
20,1.823100
40,1.520800
60,1.464300
80,1.430100
100,1.392000
120,1.359400
140,1.346400
160,1.330500
180,1.326200
200,1.323000


TrainOutput(global_step=400, training_loss=1.3543884134292603, metrics={'train_runtime': 11844.533, 'train_samples_per_second': 2.161, 'train_steps_per_second': 0.034, 'total_flos': 1.5436384637760307e+17, 'train_loss': 1.3543884134292603, 'epoch': 2.0100755667506296})

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    "./checkpoint/llama3-1b-multi-conversation/checkpoint-400",
    max_seq_length=2048,
    load_in_4bit=True,
    dtype=torch.bfloat16,
)

==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device does not support bfloat16. Will change to float16.


In [ ]:
model.push_to_hub_merged(
    "Phat-Dat/Llama-3.2-1B-Instruct-Chat-sft",
    commit_message="Merge weights to push to hub"
)

tokenizer.push_to_hub(
    "Phat-Dat/Llama-3.2-1B-Instruct-Chat-sft",
    commit_message="Push tokenizer to hub"
)